In [1]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
ratings= pd.read_csv("toy_dataset.csv",index_col=0)
ratings = ratings.fillna(0) #fill nun with 0;
# but keep 0 in place of null is not correct becoz 
#if a persion not rated the movie it doesntmean it not good

In [3]:
ratings

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,4.0,5.0,3.0,0.0,2.0,1.0
user 2,5.0,3.0,3.0,2.0,2.0,0.0
user 3,1.0,0.0,0.0,4.0,5.0,4.0
user 4,0.0,2.0,1.0,4.0,0.0,3.0
user 5,1.0,0.0,2.0,3.0,3.0,4.0


 # standardizering 

In [4]:
def standardize(row):
    new_row = (row - row.mean())/(row.max()-row.min())
    return new_row
rating_std = ratings.apply(standardize)
rating_std

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,0.36,0.6,0.400000,-0.65,-0.08,-0.35
user 2,0.56,0.2,0.400000,-0.15,-0.08,-0.60
user 3,-0.24,-0.4,-0.600000,0.35,0.52,0.40
user 4,-0.44,0.0,-0.266667,0.35,-0.48,0.15
user 5,-0.24,-0.4,0.066667,0.10,0.12,0.40


In [5]:
# we are taking transpose since we are similarity between item which need to be in rows
# we are going to compare b/w item to item w need to keep items in rows
item_similarity = cosine_similarity(rating_std.T)
print(item_similarity) # this is our mode on which we recommend 

[[ 1.          0.70668875  0.81368151 -0.79941088 -0.02539184 -0.91410609]
 [ 0.70668875  1.          0.72310153 -0.84515425 -0.5189993  -0.84337386]
 [ 0.81368151  0.72310153  1.         -0.84794611 -0.3799803  -0.80218063]
 [-0.79941088 -0.84515425 -0.84794611  1.          0.14803913  0.72374686]
 [-0.02539184 -0.5189993  -0.3799803   0.14803913  1.          0.39393939]
 [-0.91410609 -0.84337386 -0.80218063  0.72374686  0.39393939  1.        ]]


In [6]:
item_similarity = cosine_similarity(rating_std.T)
item_similarity_df = pd.DataFrame(data=item_similarity,index=ratings.columns, columns=ratings.columns)
item_similarity_df 

,action1,action2,action3,romantic1,romantic2,romantic3
action1,1.000000,0.706689,0.813682,-0.799411,-0.025392,-0.914106
action2,0.706689,1.000000,0.723102,-0.845154,-0.518999,-0.843374
action3,0.813682,0.723102,1.000000,-0.847946,-0.379980,-0.802181
romantic1,-0.799411,-0.845154,-0.847946,1.000000,0.148039,0.723747
romantic2,-0.025392,-0.518999,-0.379980,0.148039,1.000000,0.393939
romantic3,-0.914106,-0.843374,-0.802181,0.723747,0.393939,1.000000


In [7]:
## lets make recommendation 
def get_similar_movies(movie_name, user_rating):
    similar_score = item_similarity_df[movie_name] * (user_rating - 2.5)
    similar_score = similar_score.sort_values(ascending=False)
    # Convert Series to DataFrame
    return pd.DataFrame({'movie': similar_score.index, 'score': similar_score.values})
print(get_similar_movies("action1",5))

       movie     score
0    action1  2.500000
1    action3  2.034204
2    action2  1.766722
3  romantic2 -0.063480
4  romantic1 -1.998527
5  romantic3 -2.285265


In [8]:
print(get_similar_movies("romantic1",1))

       movie     score
0    action3  1.271919
1    action2  1.267731
2    action1  1.199116
3  romantic2 -0.222059
4  romantic3 -1.085620
5  romantic1 -1.500000


In [9]:
## but see here is the problem comes ,when you rate 1 for romantic move 
## even then it recommanding you romantic movies it not correct.we need to send all similar movesto down
## we do a small correction in recommand function 

In [10]:
def get_similar_movies(movie_name, user_rating):
    similar_score = item_similarity_df[movie_name] * (user_rating - 2.5)
    similar_score = similar_score.sort_values(ascending=False)
    # Convert Series to DataFrame
    return pd.DataFrame({'movie': similar_score.index, 'score': similar_score.values})
print(get_similar_movies("action1",5))

       movie     score
0    action1  2.500000
1    action3  2.034204
2    action2  1.766722
3  romantic2 -0.063480
4  romantic1 -1.998527
5  romantic3 -2.285265


In [11]:
print(get_similar_movies("romantic1",1))

       movie     score
0    action3  1.271919
1    action2  1.267731
2    action1  1.199116
3  romantic2 -0.222059
4  romantic3 -1.085620
5  romantic1 -1.500000


In [12]:

def get_similar_movies(movie_name, user_rating):
    similar_score = item_similarity_df[movie_name] * (user_rating - 2.5)
    similar_score = similar_score.sort_values(ascending=False)
    # Convert Series to DataFrame
    return pd.DataFrame({'movie': similar_score.index, 'score': similar_score.values})

action_lover = [("action1", 5), ("romantic2", 1), ("romantic3", 1)]
similar_movies_list = []

for movie, rating in action_lover:
    new_movies = get_similar_movies(movie, rating)
    similar_movies_list.append(new_movies)

# Concatenate all DataFrames at once
similar_movies = pd.concat(similar_movies_list, ignore_index=True)

# Aggregate scores by movie name, summing the scores where movies repeat
aggregated_scores = similar_movies.groupby('movie').score.sum().reset_index()

# Sort the aggregated scores DataFrame by score in descending order
sorted_scores = aggregated_scores.sort_values(by='score', ascending=False)

# Print the sorted scores DataFrame
print(sorted_scores)



       movie     score
0    action1  3.909247
1    action2  3.810282
2    action3  3.807445
4  romantic2 -2.154389
3  romantic1 -3.306206
5  romantic3 -4.376174


In [13]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'